In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn as nn
import torch
import itertools

from emnist import get_emnist_data_loaders
from models.crnn.crnn import CRNN
from models.lstm_cnn.lstm_cnn import CNN_LSTM
from training.trainer import Trainer
from hyperparameter import common_hyperparameters, model_specific_hyperparameters
from evaluation.loss import plot_loss
from evaluation.evaluations_utils import create_directory_if_not_exists
from models.crnn.crnn import CRNN
from models.lstm_cnn.lstm_cnn import CNN_LSTM
import torch.nn as nn
from training.trainer import Trainer
import hyperparameter as hp
import torch
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ExponentialLR, CosineAnnealingLR, LambdaLR, CyclicLR

In [3]:
train_loader, val_loader, test_loader = get_emnist_data_loaders(hp.NUMBER_OF_SEQUENCES, hp.DIGITS_PER_SEQUENCE, hp.BATCH_SIZE, hp.BATCH_SIZE_VAL)

TypeError: get_emnist_data_loaders() missing 4 required positional arguments: 'number_of_sequences', 'digits_per_sequence', 'batch_size', and 'batch_size_val'

In [ ]:
# Generate all possible combinations of hyperparameters
param_combinations = list(itertools.product(*model_specific_hyperparameters.values()))
print(f"Number of Hyperparameter combinations: {len(param_combinations)}")

In [ ]:
# Define your models
models = {
    "crnn": CRNN(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT),
    "lstm_cnn": CNN_LSTM(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)
}

# Define your criteria
criteria = {
    "crnn": nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True),
    "lstm_cnn": nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)
}

# Create dictionaries for optimizers and schedulers
optimizer_configs = {
    "adam": torch.optim.Adam,
    # "adam_w": torch.optim.AdamW,
    "adam_w_regulise": torch.optim.AdamW
}

scheduler_configs = {
    "step_lr": (StepLR, {"step_size": 5, "gamma": 0.5}),
    # "multi_step_lr": (MultiStepLR, {"milestones": [5, 10, 15], "gamma": 0.5}),
    "exponential_lr": (ExponentialLR, {"gamma": 0.95}),
    # "cosine_annealing_lr": (CosineAnnealingLR, {"T_max": 10}),
    # "lambda_lr": (LambdaLR, {"lr_lambda": lambda epoch: 0.95 ** epoch}),
    # "cyclic_lr": (CyclicLR, {"base_lr": 0.001, "max_lr": 0.1})
}

# Initialize an empty dictionary to store trainers
trainers = {}

# Iterate through models, criteria, optimizers, and schedulers to create trainers
for model_name, model in models.items():
    for optimizer_name, optimizer_class in optimizer_configs.items():
        if optimizer_name == "adam_w_regulise":
            optimizer = optimizer_class(model.parameters(), lr=hp.LEARNING_RATE, weight_decay=hp.WEIGHT_DECAY)
        else:
            optimizer = optimizer_class(model.parameters(), lr=hp.LEARNING_RATE)
        for scheduler_name, (scheduler_class, scheduler_args) in scheduler_configs.items():
            scheduler = scheduler_class(optimizer, **scheduler_args)
            criterion = criteria[model_name]
            trainer_name = f"{model_name}_{optimizer_name}_{scheduler_name}_trainer"
            trainer = Trainer(
                model, model_name, criterion, optimizer, train_loader, val_loader, test_loader,
                epochs=hp.EPOCHS, args={"blank_label": hp.BLANK_LABEL, "cnn_output_width": hp.CNN_OUTPUT_WIDTH}
            )
            trainers[trainer_name] = trainer
print(f"Prepared {len(trainers)} models for training.")


NameError: name 'train_loader' is not defined

In [ ]:
metrics_global = {}
for model_name, trainer in trainers.items():
    print(f"Training {model_name}")
    metrics = trainer.train_validate_test()
    metrics_global[model_name] = metrics
    print(f"Finished training {model_name}")

    # Save the trained model
    torch.save(trainer.model.state_dict(), 'models/' + model_name + '.pth')
    break

Training crnn_adam_step_lr_trainer
100%|██████████| 110/110 [00:31<00:00,  3.55it/s]
EPOCH 1/10 - TRAINING. Correct: 0/7000 = 0.0000 - Average CER Score: 0.984
100%|██████████| 2999/2999 [00:14<00:00, 211.47it/s]
EPOCH 1/10 - VALIDATING. Correct: 1/2999 = 0.0003 - Average CER Score: 0.774
100%|██████████| 110/110 [00:30<00:00,  3.56it/s]
EPOCH 2/10 - TRAINING. Correct: 3540/7000 = 0.5057 - Average CER Score: 0.602
100%|██████████| 2999/2999 [00:14<00:00, 208.41it/s]
EPOCH 2/10 - VALIDATING. Correct: 2238/2999 = 0.7462 - Average CER Score: 0.424
100%|██████████| 110/110 [00:30<00:00,  3.57it/s]
EPOCH 3/10 - TRAINING. Correct: 5663/7000 = 0.8090 - Average CER Score: 0.419
100%|██████████| 2999/2999 [00:14<00:00, 207.60it/s]
EPOCH 3/10 - VALIDATING. Correct: 2450/2999 = 0.8169 - Average CER Score: 0.298
100%|██████████| 110/110 [00:30<00:00,  3.55it/s]
EPOCH 4/10 - TRAINING. Correct: 6101/7000 = 0.8716 - Average CER Score: 0.322
100%|██████████| 2999/2999 [00:14<00:00, 206.94it/s]
EPOCH 4

KeyboardInterrupt: 

In [ ]:
from evaluation.loss import plot_loss
for model_name, trainer in trainers.items():
    # Load model

    base_path = 'models/'
    model_path = model_name + '.pth'
    model = trainer.model.to("cpu")
    trainer.model.load_state_dict(torch.load(model_path))

    # Plot the trained model
    plot_loss(model_name=model_name, train_loss_per_epoch=trainer.epoch_train_cers, val_loss_per_epoch=trainer.epoch_val_cers, figure_evaluation_dir=base_path)
    break

FileNotFoundError: [Errno 2] No such file or directory: 'crnn_adam_step_lr_trainer.pth'

In [ ]:
for model_name, trainer in trainers.items():
    trainer.load_model(f'models/{model_name}/model.pth')
    trainer.test(plot_n=1)
    break

100%|██████████| 1/1 [00:00<00:00, 184.91it/s]


AttributeError: 'Trainer' object has no attribute 'epoch_test_cers'